In [2]:
from sentence_transformers import CrossEncoder
import pandas as pd
from tqdm import tqdm
model = CrossEncoder('cross-encoder/nli-distilroberta-base')

/root/anaconda3/envs/SBERT-WK/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 701/701 [00:00<00:00, 676kB/s]
Downloading: 100%|██████████| 313M/313M [00:39<00:00, 8.23MB/s]
Downloading: 100%|██████████| 25.0/25.0 [00:00<00:00, 14.4kB/s]
Downloading: 100%|██████████| 878k/878k [00:01<00:00, 625kB/s]
Downloading: 100%|██████████| 446k/446k [00:01<00:00, 383kB/s]
Downloading: 100%|██████████| 772/772 [00:00<00:00, 365kB/s]


In [17]:
def nli(sen1,sen2):
    scores = model.predict([(sen1,sen2)])
    label_mapping = ['contradiction', 'entailment', 'neutral']
    labels = [label_mapping[score_max] for score_max in scores.argmax(axis=1)]
    return labels




In [15]:
nli("Ted Cruz said that he literally talks to God", "Ted Cruz never said that he literally talks to God.")

'contradiction'

In [16]:
df = pd.read_json('/root/thesis/ViLT/cosmos/test_data.json',orient="records", lines=True)

In [18]:
tqdm.pandas()
df['nli_label'] = df.progress_apply(lambda x: nli(x.caption1,x.caption2)[0], axis=1)

100%|██████████| 1700/1700 [00:26<00:00, 63.78it/s]


In [19]:
df

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,nli_label
0,test/0.jpg,Julian Castro at his announcement in San Anton...,Julian Castro at his announcement in San Anton...,0,https://www.nytimes.com/2019/06/13/us/politics...,"[[389.9706726074219, 72.9228744506836, 505.056...","PERSON at his announcement in GPE, GPE, on DAT...","[[Julian Castro, PERSON], [San Antonio, GPE], ...","PERSON at his announcement in GPE, GPE, on DATE.","[[Julian Castro, PERSON], [San Antonio, GPE], ...",0.576995,0.601183,neutral
1,test/1.jpg,Supporters of Tanzania's ruling Chama Cha Mapi...,A person sits on a truck as supporters of the ...,0,https://www.bbc.com/news/world-africa-54828934,"[[389.6280517578125, 8.949727058410645, 609.61...",Supporters of GPE's ruling ORG party come out ...,"[[Tanzania, GPE], [Chama Cha Mapinduzi, ORG], ...",A person sits on a truck as supporters of the ...,"[[Chama Cha Mapinduzi, PERSON], [Revolutionary...",0.541939,0.729243,contradiction
2,test/2.jpg,"53,000 dead people turned up on the state’s vo...",These social media posts did not link to a rec...,1,https://www.snopes.com/fact-check/53000-dead-f...,"[[0.0, 14.214579582214355, 326.70501708984375,...",CARDINAL dead people turned up on the state’s ...,"[[53,000, CARDINAL], [November 2018, DATE]]",These social media posts did not link to a rec...,"[[Florida, GPE], [November 2018, DATE], [6-yea...",0.234810,0.307628,neutral
3,test/3.jpg,"Actor, musician, director and devoted follower...",A shocking report about the former child actor...,1,https://www.snopes.com/fact-check/kirk-cameron...,"[[491.4454345703125, 361.714599609375, 538.475...","Actor, musician, director and devoted follower...","[[Christ, ORG], [Kirk Cameron, PERSON]]",A shocking report about the former child actor...,"[[Kirk Cameron, PERSON]]",0.460771,0.425086,neutral
4,test/4.jpg,Men from the Maasai tribe perform a traditiona...,"And on the same day in Kenya's Narok county, y...",0,https://www.bbc.com/news/world-africa-56278224,"[[194.4141845703125, 31.10932731628418, 381.77...",Men from the GPE tribe perform a traditional j...,"[[Maasai, GPE], [Masai-Mara, ORG]]","And on DATE in GPE's GPE, young PERSON men tak...","[[the same day, DATE], [Kenya, GPE], [Narok co...",0.619695,0.675563,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,test/1695.jpg,President Obama trademarked the name 'Obamacar...,There was no truth that Obama family millions ...,1,https://www.snopes.com/fact-check/obamacare-ro...,"[[283.2491455078125, 28.175857543945312, 700.4...",President PERSON trademarked the name 'ORG' an...,"[[Obama, PERSON], [Obamacare, ORG]]",There was no truth that PERSON family MONEY wi...,"[[Obama, PERSON], [millions of dollars, MONEY]]",0.477130,0.506001,neutral
1696,test/1696.jpg,A photograph shows a soldier carrying a donkey...,Coronavirus meme featuring “WWII donkey” is no...,1,https://www.snopes.com/fact-check/soldier-carr...,"[[283.0551452636719, 82.37718200683594, 378.09...",A photograph shows a soldier carrying a donkey...,"[[World War II, EVENT]]",ORG meme featuring “EVENT donkey” is not what ...,"[[Coronavirus, ORG], [WWII, EVENT]]",0.489925,0.387084,neutral
1697,test/1697.jpg,Homeless people living on streets in Denver,The State Capitol Building in Colorado,0,https://www.denverpost.com/2020/01/06/denver-c...,"[[477.35260009765625, 299.9366760253906, 524.9...",Homeless people living on streets in GPE,"[[Denver, GPE]]",The State Capitol Building in GPE,"[[Colorado, GPE]]",0.185943,0.406328,contradiction
1698,test/1698.jpg,The castle's esplanade was a perfect spot for ...,Picture shows an Edinburgh skier,0,https://www.bbc.com/news/uk-scotland-55751229,"[[395.11273193359375, 187.15399169921875, 561....",The castle's esplanade was a perfect spot for ...,[],Picture shows an PERSON skier,"[[Edinburgh, PERSON]]",0.547756,0.522018,neutral


In [22]:
converter = lambda x : 0 if x.nli_label == 'entailment' else 1
df['predict'] =df.progress_apply(converter, axis=1) 
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

100%|██████████| 1700/1700 [00:00<00:00, 48141.06it/s]

Actual       0    1
Predicted          
0          187   11
1          663  839
accuracy: 0.6035294117647059
